In [ ]:
import os
os.chdir("../")


In [ ]:
%config Completer.use_jedi = False



In [ ]:
import yaml
import torch
import random
import argparse
import numpy as np
import matplotlib
import pandas as pd
from src.dataset_hrg import getDataLoader
from src.module_hrg import EmbeddingHRG
from torch_geometric.data.batch import Batch
from torch.optim import AdamW
import os
%load_ext autoreload
%autoreload 2


In [ ]:
## Classifier
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import torch
import torch.nn as nn

class DurationClassifier(torch.nn.Module):
    def __init__(self, h_dim, n_buckets):
        super(DurationClassifier, self).__init__()
        self.gcn = EmbeddingHRG(data_tr.dataset.n_vocab, self.h_dim, self.h_dim).cuda()
        self.n_buckets = n_buckets
        self.h_dim = h_dim
        self.cls_head = nn.Sequential(nn.Linear(self.h_dim, self.h_dim // 2),
                                      nn.ReLU(),
                                      nn.Linear(self.h_dim // 2, self.n_buckets))
    def forward(self, x):
        x = self.gcn(x)
        return self.cls_head(x)
        
        

### Step 1: Load dataset


In [ ]:
config = yaml.load(open("config/config-arctic.yaml", 'r'), Loader=yaml.FullLoader)
_config = config['solver']

In [ ]:
data_tr = getDataLoader(
    mode='train',
    meta_path=_config['meta_path']['train'],
    data_dir=_config['data_dir'],
    batch_size=_config['batch_size'],
    r=config['model']['tacotron']['r'],
    n_jobs=_config['n_jobs'],
    use_gpu=True,
    add_info_headers=["dur"])

In [ ]:
n_buckets = 11
h_dim = 256
label_dict = {f"DB_{i}": i for i in range(n_buckets)}

### Step 2: Init the model

In [ ]:
dur_classifier = DurationClassifier(h_dim=h_dim, n_buckets=n_buckets).cuda()

In [ ]:
optim = AdamW(dur_classifier.parameters())

In [ ]:
def parse_labels(label_seq):
    label_seq = [ls.strip().split() for ls in label_seq]
    res = []
    for ls in label_seq:
        res.append([label_dict[x] for x in ls])
    return res
    

In [ ]:
loss_f = nn.CrossEntropyLoss(ignore_index=-1)

In [ ]:
for batch in data_tr:
    # for a batch, run GCN
    _, txt, text_lengths, mel, spec, add_info = batch
    add_info = parse_labels(add_info)
    output = model(txt)
    bsz, max_nodes, hsz = output.shape; bsz, max_nodes, hsz
    
    # duration classification
    class_output = dur_classifier(output); 
    
    # get a representation for the syll nodes
    syll_dur = torch.zeros(len(add_info), max_nodes).long() - 1
    for i in range(len(add_info)):
        syll_dur[i, :len(add_info[i])] = torch.LongTensor(add_info[i])
    loss_val = loss_f(class_output.view(-1, n_buckets), syll_dur.view(-1).cuda())
    print(loss_val)

In [ ]:
class_output.shape

### Add duration

In [ ]:
meta_all = pd.read_csv("arctic/meta_all.txt", sep="|", names=["mel", "lin", "len", "hrg", "add_info"])
meta_all = meta_all.drop(labels="add_info", axis=1)
syll_dur = pd.read_csv("syll_dur_end.txt", sep="|", names=["source", "target", "end"])
merged = pd.concat([meta_all, syll_dur], axis=1)
merged[["mel", "lin", "len", "hrg", "target"]].to_csv("arctic/meta_all.txt", sep="|", header=False, index=None)